In [ ]:
from __future__ import print_function
import Hubbard.hamiltonian as hh
import Hubbard.geometry as geometry
import Hubbard.sp2 as sp2
import Hubbard.density as density
from Hubbard.negf import NEGF
import Hubbard.plot as plot
%matplotlib inline

In this example we will study the electron correlations in an open-quantum system, and find the self-consistent solution using the non-equilibrium Green's function formalism within the `Hubbard` package.

We will find the transmission probabilities per spin channel.
Look into example [TB_04](https://github.com/zerothi/ts-tbt-sisl-tutorial/blob/master/TB_04/run.ipynb) for more information about how [TBtrans](https://launchpad.net/siesta) and [sisl](https://sisl.readthedocs.io/en/v0.10.0/api.html) can be combined.

The sysem of this example is composed by a central region coupled to two electrodes (left and right). The central region in this case will be just a repetition of the unit-cell of the electrodes (perfect system), for the sake of simplicity. We will consider the electrodes to be the unit cell of a zigzag graphene nanoribbon (ZGNR) of width W=5 C-atoms across periodic along the x-axis.

We will focus on the equilibrium situation, therefore the temperature and chemical potentials of the electrodes *must coincide*. The complex contour that we use to integrate the density matrix in the `Hubbard.NEGF` class is extracted from a [Transiesta](https://launchpad.net/siesta) calculation performed for a temperature of `kT=0.025` eV, which we will set as common for all the composing element calculations.

In [ ]:
# Set U and kT for the whole calculation
U = 2.0
kT = 0.025

- We will start by building the tight-binding (TB) Hamiltonian for the graphene nanoribbons, which compose the electrodes (periodic boundary conditions). Then we will find their self-consistent solution by using the `Hubbard` package.

In [ ]:
# Build sisl.Geometry object of a ZGNR of width W=5 C-atoms across,
# e.g., by using the function Hubbard.geometry.zgnr.
# This function returns a periodic ZGNR along the x-axis.
ZGNR = geometry.zgnr(5)

# Build tight-binding Hamiltonian using sp2 function
H_elec = sp2(ZGNR, t1=2.7, t2=0.2, t3=0.18, s1=0, s2=0, s3=0)

# Build the HubbardHamiltonian object for the periodic system:
MFH_elec = hh.HubbardHamiltonian(H_elec, U=U, kT=kT, nkpt=[100, 1, 1])

# Initialize electrodes spin density
MFH_elec.set_polarization([0], dn=[9])

# Converge Electrode Hamiltonians
dn = MFH_elec.converge(density.dm, tol=1e-10, print_info=True)

- Let's plot the spin polarization to check out the self-consistent solution

In [ ]:
# Now we have the converged electrodes saved in the MFH_elec variable
p = plot.SpinPolarization(MFH_elec, colorbar=True, vmax=0.4, vmin=-0.4)

In [ ]:
# Build central region TB Hamiltonian
HC = H_elec.tile(16, axis=0)

# Remove atoms in the central area to recreate the system of Fig. 3 (b) of the reference paper.
HC = HC.remove([69, 79, 89, 78, 88, 66, 77, 76, 87, 86, 65, 74, 75, 84, 85])

# Make it finite
HC.set_nsc([1, 1, 1])

# Write geometry to file if one wants to visualize it
HC.geometry.write('device.xyz')